In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [18]:
examples = [
    {
        "country": "France",
        "answer" : """
        Here is what I know:
        Capital: Paris
        Language: France
        Food: Wine and Cheese
        Currency: Euro
        """ 
    },
    {
        "country": "Italy",
        "answer" : """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

In [22]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=10,
)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)
# example_prompt = ChatPromptTemplate.from_messages([
#     ("human", "What do you know about {country}?"),
#     ("ai", "{answer}")
# # ])
# prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples
# )
# final_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a geography expert, you give short answers."),
#     example_prompt,
#     ("human", "What do you know about {country}?")
# ])

# chain = final_prompt | chat
# prompt.format(country="Germany")

# chain = prompt | chat
# chain.invoke({
#     "country":"Germany"
# })
